# Importações

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("./dados_tratados.csv",sep=",")

In [ ]:
data.columns

# Clusterização

In [ ]:
from sklearn.cluster import KMeans

Calculo para o numero de clusters

In [ ]:
data.shape[0]/(400*22)

In [ ]:
X=data.iloc[:,1:3].values

In [ ]:
X.shape

Instacia do objeto kmeans

In [ ]:
n_dia = 400
dias =22
kmeans = KMeans(n_clusters=int(data.shape[0]/(n_dia*dias)))

Treinamento com os dados de longitude e latitude

In [ ]:
kmeans.cluster_centers_

In [ ]:
kmeans.labels_

In [ ]:
kmeans.fit(X)

Atribuição dos clusters aos dados

In [ ]:
pred =kmeans.predict(X)

In [ ]:
pred.shape

Criação dos clusters de leituristas no dataframe

In [ ]:
data["cluster_leiturista"] = pred

Criação dos clusters de dias para cada leiturista/cluster

In [ ]:
dias_leitura = 22
data['cluster_dia'] = 0
for i in np.unique(kmeans.labels_):
  X = data.loc[data["cluster_leiturista"]==i].iloc[:,1:3].values
  kmeans2 = KMeans(n_clusters=dias_leitura)
  pred2=kmeans2.fit_predict(X)
  data.loc[data["cluster_leiturista"]==i,'cluster_dia']=pred2


In [ ]:
def balancing_kmeans_day(data:pd.DataFrame, centroids_coords,ponts_qtd):
    conflicts = []
    grouped=data.groupby('cluster_leiturista')
    
    conflicts_qtd =0
    qtd_clusters=0
    
    for _, group in grouped:
    
        clusters = group.groupby('cluster_dia')
        solution_leiturista = []
        solution_distance = 0


        for label, cluster in clusters:
            qtd_clusters+=1
            qtd_points = cluster.shape[0]
            if qtd_points<(ponts_qtd - 0.05*ponts_qtd) or qtd_points>(ponts_qtd + 0.5*ponts_qtd):
                conflicts.append(cluster)
                conflicts_qtd+=1
    print(conflicts_qtd)
    print(qtd_clusters)

            



In [ ]:
for i, j in enumerate(kmeans.cluster_centers_):
    print(i)
    print(j)

In [ ]:
import random

In [ ]:
from haversine import haversine, Unit


In [ ]:
def verify_excess(data,days,ponts_qtd):
    grouped=data.groupby('cluster_leiturista')

    for _, group in grouped:
        qtd_points = group.shape[0]
        if qtd_points/days > (ponts_qtd +50):
            if qtd_points/days - ponts_qtd >0.5*ponts_qtd:
                return True
    return False
    

In [ ]:
def balancing_kmeans_leit(data:pd.DataFrame, centroids_coords,ponts_qtd,days):
    grouped=data.groupby('cluster_leiturista')
    excess =0
    lack =0
    lack_list = []
    excess_list = []
    excess_index =[]
    conflicts_qtd =0
    qtd_clusters=0
    excess_gap=0
    lack_gap = 0
    lack_index=[]
    clusters_proximity = []
    trocas = []
    for _, group in grouped:
    
        clusters = group.groupby('cluster_dia')
        solution_leiturista = []
        add_cl=0
        solution_distance = 0
        qtd_clusters+=1
        qtd_points = group.shape[0]
        if qtd_points/days < (ponts_qtd-10) :
            lack+=1
            lack_list.append({"cluster":group['cluster_leiturista'].values[0],"lack_per_day":abs(qtd_points/days - ponts_qtd) ,"points_quantity":qtd_points/22})
            lack_index.append(group['cluster_leiturista'].values[0])
            lack_gap += abs(qtd_points/days - ponts_qtd)
            if abs(qtd_points/days - ponts_qtd) >ponts_qtd:
                lack_list.append(group['cluster_leiturista'].values[0])
                print(f'ponts_gaps:{qtd_points/days - ponts_qtd}')
                print(f'pointsqtd: {qtd_points/days}')
        if qtd_points/days > (ponts_qtd +50):
            if qtd_points/days - ponts_qtd >0.5*ponts_qtd:
                print(qtd_points)
                excess+=1
                excess_list.append({"cluster":group['cluster_leiturista'].values[0],"excess_per_day":qtd_points/days - ponts_qtd ,"points_quantity":qtd_points/22})
                excess_index.append(group['cluster_leiturista'].values[0])

                kmeans3 =KMeans(n_clusters=2)
                clusters_index = group["cluster_leiturista"].values[0]
                group_data = data.loc[data["cluster_leiturista"]==clusters_index].iloc[:,2:4].values
                print(clusters_index)
                print(data["cluster_leiturista"].unique())
                group_coords =group.iloc[:,2:4].values
                print(group_coords)
                kmeans3.fit(group_coords)
                pred = kmeans3.predict(group_data)
                pred = pred+100 + int(10*random.random())
                print(np.unique(pred,return_counts=True))
                print(pred)
                data.loc[data["cluster_leiturista"]==clusters_index,'cluster_leiturista'] = pred
                for i in np.unique(pred):
                    X = data.loc[data["cluster_leiturista"]==i].iloc[:,2:4].values
                    kmeans4 = KMeans(n_clusters=days)
                    pred2=kmeans4.fit_predict(X)
                    data.loc[data["cluster_leiturista"]==i,'cluster_dia']=pred2

                print(f'ponts_gaps:{qtd_points/days - ponts_qtd}')
                print(f'pointsqtd: {qtd_points/days}')
                excess_list.remove(excess_list[-1])
                excess_index.remove(excess_index[-1])
                
            excess_gap += abs(qtd_points/days - ponts_qtd)

    for cluster, coords in enumerate(kmeans.cluster_centers_):
        coords_distances = []
        for cluster2, coords2 in enumerate(kmeans.cluster_centers_):
            if cluster != cluster2:
                dist = haversine(coords, coords2, unit=Unit.KILOMETERS)
                coords_distances.append({"cluster_":cluster2,"distance":dist})
        coords_distances.sort(key=lambda x: x["distance"])
        clusters_proximity.append({"cluster_origin":cluster,"clusters_proximities":coords_distances})
    clusters_proximity.sort(key= lambda x:x["cluster_origin"])

    for i in excess_list:
        proximites = clusters_proximity[i["cluster"]]
        for j in proximites["clusters_proximities"]:
            if j["cluster_"] in lack_index:
                lack = lack_index.index(j["cluster_"])
                print(f'distance between {proximites["cluster_origin"]},{j["cluster_"]} : {j["distance"]}  ')
                print(f'excess: {i["excess_per_day"]}')

                print(f'lack: {lack_list[lack]["lack_per_day"]}')
                break


    


    print(f'excess_qtd:{excess}')
    print(f'lack_qtd:{lack}')
    print(f'excess_gap:{excess_gap}')
    print(f'excess_gap_in_leit:{excess_gap/days}')
    print(f'lack_list:{lack_list}')
    print(f'excess_list:{excess_list}')
    print(f'lack_gap:{lack_gap}')
    print(clusters_proximity)
    print(clusters_proximity[4])
    

            



In [ ]:
balancing_kmeans_leit(data,0,400,22)

In [ ]:
dt = data.loc[(data["cluster_leiturista"]== 100 )|(data["cluster_leiturista"]== 101 )] 

In [ ]:
dt.shape[0]/22

In [ ]:
data = pd.read_csv('dados_tratados_clusterizados_400.csv')
balancing_kmeans_leit(data,0,400,22)

In [ ]:
data["cluster_leiturista"].unique()

In [ ]:
sns.scatterplot(x=dt['LONGITUDE'],y=dt['LATITUDE'],hue = dt['cluster_leiturista'], palette=sns.color_palette("Spectral", as_cmap=True))
sns.scatterplot(x=clis[:,1],y=clis[:,0])


In [ ]:
14837/22

In [ ]:
data.loc[data["cluster_leiturista"]==103].shape[0]/22

In [ ]:
data["cluster_leiturista"].unique().shape

In [ ]:
balancing_kmeans_leit(data,0,200)

verificaçãodo dataframe novo para o cluster de leiturista 1

In [ ]:
data.loc[data["cluster_leiturista"]==1]

In [ ]:
clis = kmeans.cluster_centers_

In [ ]:
import seaborn as sns

Plot dos clusters de leituristas para todos os dados

In [ ]:
dia_ex = data.loc[data["cluster_leiturista"]==10]

plot dos clusters de dias do leiturista de numero 10

In [ ]:
sns.scatterplot(data = dia_ex,x="LONGITUDE",y="LATITUDE",hue="cluster_dia")

In [ ]:
data.to_csv("./dados_tratados_clusterizados_400.csv",sep=",",encoding='utf-8')

In [ ]:
from haversine import haversine, Unit

grouped=data.groupby('cluster_leiturista')
solutions =[]
for _, group in grouped:
   
    clusters = group.groupby('cluster_dia')
    solution_leiturista = []
    solution_distance = 0

    for label, cluster in clusters:
        coords = cluster[['LATITUDE', 'LONGITUDE']].to_numpy()   


        n = len(coords)
        distance_matrix = np.zeros((n, n))
        qtd_matrix = cluster['QUANTIDADE_HIDROMETROS'].values
        qtd_matrix=np.insert(qtd_matrix,[0],[0],axis=0)
        for i in range(n):
            for j in range(i + 1, n):
                distance_matrix[i, j] = distance_matrix[j, i] = haversine(coords[i], coords[j], unit=Unit.KILOMETERS)

        